# DB_CODE| DATA MIGRATION FROM OLD DB TO NEW DB (AWS)
---
---

# Imports

#### Standard library imports

In [38]:
import sys
sys.path.append("../../")

#### Third party imports

In [2]:
import pandas as pd

#### Local application imports

In [39]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *
from pkg_dir.src.utils.sql_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extracting data

In [4]:
## Function parameters
db_crds = 'pc_db_prod'
sql_files_path = '../../pkg_dir/sql/'
sql_script = 'appointments.sql'
sql_params = sql_params_appointments

In [5]:
dfx = sql_to_df(db_crds, sql_files_path, sql_script, sql_params)

# Manipulating data

In [7]:
## Reference to simplify labels in appointment status
values_map = {
    
    'COMPLETADA': 'completed',
    'SALA_ESPERA': 'completed',
    'CONSULTA': 'completed',
    'CONFIRMADO_PAGO': 'completed',

    'NO_PRESENTO': 'no_show',
    'MENSAJE_DEJADO': 'no_show',
    'NO_CONFIRMADO': 'no_show',
    'CONFIRMADO': 'no_show',
    'LLEGA_TARDE_ESPERA': 'no_show',
    'REAGENDAR': 'no_show',
    
}

In [8]:
## Mapping values according to reference
dfx['appointment_status_simple'] = dfx['appointment_status'].map(values_map).fillna(dfx['appointment_status'])

In [21]:
## Function parameters for AWS DATABASE
aws_db_crds = 'aws_db'
sql_files_path = '../../pkg_dir/sql/'
sql_script = 'aws_test.sql'
aws_df = dfx.head(10)
input_data = aws_df.values

In [40]:
## Drop table if exists
conn = database_conection(aws_db_crds)
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS raw;')
cur.close()
conn.commit()

In [41]:
## AWS migration function
migrate_to_aws(aws_db_crds, sql_files_path, sql_script, input_data)

'Completed'

In [43]:
## Testing to see if data was uploaded to AWS properly
conn = database_conection(aws_db_crds)
cur = conn.cursor()
cur.execute('SELECT * FROM raw;')
return_df = pd.DataFrame(cur.fetchall())

In [44]:
aws_df

,appointment_id,appointment_date,appointment_start_time,appointment_end_time,appointment_creation,appointment_status,doctor,medical_specialty,clinic,patient_id,patient_birth_date,appointment_status_simple
0,2023000581549,2023-11-15,14:00,14:30,12/11/2023 12:54:18,NO_PRESENTO,DR. Fernando Iván Vergara Orduña,CARDIOLOGIA,MONTEVIDEO,92586,1953-11-28,no_show
1,2023000854685,2023-11-14,16:00,16:30,12/11/2023 15:51:52,COMPLETADA,DRA KARLA VANESSA RODRIGUEZ JIMENEZ,NEUROLOGIA,DEL VALLE,272735,1938-01-28,completed
2,2023001165360,2023-11-15,10:30,11:00,12/11/2023 17:02:45,COMPLETADA,DRA ABIGAIL GONZALEZ MORALES,DERMATOLOGIA,DOCTORES,104330,1999-02-22,completed
3,2023000999514,2023-11-13,11:30,12:00,13/11/2023 07:15:22,NO_PRESENTO,DRA NAYANSI CABRERA GARCIA,GINECOLOGIA,DOCTORES,272779,2003-12-10,no_show
4,2023000648833,2023-11-22,07:30,08:00,13/11/2023 09:11:24,COMPLETADA,DRA Perla Lourdes Díaz Aguirre,ANGIOLOGIA,BASILICA,272826,1956-10-07,completed
5,2023001186934,2023-11-20,12:10,12:50,13/11/2023 09:33:12,COMPLETADA,DRA LUZ MARIA PINEDA CENTENO,NEUROLOGIA,TLALPAN,215593,1958-10-28,completed
6,2023001043445,2023-11-13,11:30,12:00,13/11/2023 09:51:39,COMPLETADA,DR. FRANCISCO ALVARADO BELTRAN,ORTOPEDIA,DOCTORES,272857,1968-07-09,completed
7,2023000887515,2023-11-22,17:30,18:00,13/11/2023 10:04:46,CONSULTA,DR. OMAR ANTONY GOMEZ MORALES,Ultrasonido,DEL VALLE,272771,1993-12-08,completed
8,2023001040935,2023-11-13,18:00,18:30,13/11/2023 10:23:26,CONSULTA,DR. CARLOS ANDRES CAMPOVERDE ESPINOZA,GASTROENTEROLOGIA,DOCTORES,269388,1995-12-31,completed
9,2023000446422,2023-11-14,10:00,11:00,13/11/2023 10:30:17,COMPLETADA,DR. Christopher Davalos Carbajal,UROLOGIA,BASILICA,236167,1965-07-27,completed


In [45]:
return_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,2023000581549,2023-11-15,14:00,14:30,12/11/2023 12:54:18,NO_PRESENTO,DR. Fernando Iván Vergara Orduña,CARDIOLOGIA,MONTEVIDEO,92586,1953-11-28,no_show
1,2023000854685,2023-11-14,16:00,16:30,12/11/2023 15:51:52,COMPLETADA,DRA KARLA VANESSA RODRIGUEZ JIMENEZ,NEUROLOGIA,DEL VALLE,272735,1938-01-28,completed
2,2023001165360,2023-11-15,10:30,11:00,12/11/2023 17:02:45,COMPLETADA,DRA ABIGAIL GONZALEZ MORALES,DERMATOLOGIA,DOCTORES,104330,1999-02-22,completed
3,2023000999514,2023-11-13,11:30,12:00,13/11/2023 07:15:22,NO_PRESENTO,DRA NAYANSI CABRERA GARCIA,GINECOLOGIA,DOCTORES,272779,2003-12-10,no_show
4,2023000648833,2023-11-22,07:30,08:00,13/11/2023 09:11:24,COMPLETADA,DRA Perla Lourdes Díaz Aguirre,ANGIOLOGIA,BASILICA,272826,1956-10-07,completed
5,2023001186934,2023-11-20,12:10,12:50,13/11/2023 09:33:12,COMPLETADA,DRA LUZ MARIA PINEDA CENTENO,NEUROLOGIA,TLALPAN,215593,1958-10-28,completed
6,2023001043445,2023-11-13,11:30,12:00,13/11/2023 09:51:39,COMPLETADA,DR. FRANCISCO ALVARADO BELTRAN,ORTOPEDIA,DOCTORES,272857,1968-07-09,completed
7,2023000887515,2023-11-22,17:30,18:00,13/11/2023 10:04:46,CONSULTA,DR. OMAR ANTONY GOMEZ MORALES,Ultrasonido,DEL VALLE,272771,1993-12-08,completed
8,2023001040935,2023-11-13,18:00,18:30,13/11/2023 10:23:26,CONSULTA,DR. CARLOS ANDRES CAMPOVERDE ESPINOZA,GASTROENTEROLOGIA,DOCTORES,269388,1995-12-31,completed
9,2023000446422,2023-11-14,10:00,11:00,13/11/2023 10:30:17,COMPLETADA,DR. Christopher Davalos Carbajal,UROLOGIA,BASILICA,236167,1965-07-27,completed
